# Get all open questions

In [1]:
import load_secrets
load_secrets.load_secrets()

In [2]:
import forecasting_tools

In [3]:
from forecasting_tools import MetaculusApi, ApiFilter, DataOrganizer
from datetime import datetime
import asyncio, time, joblib

In [4]:
api_filter = ApiFilter(
    allowed_statuses=["open"],
)

all_open = []

In [5]:
for chunk in range(0,1000):
    print('chunk starting', chunk)
    offset = chunk * MetaculusApi.MAX_QUESTIONS_FROM_QUESTION_API_PER_REQUEST
    questions, success = MetaculusApi._grab_filtered_questions_with_offset(api_filter, offset)
    if success:
        all_open.extend(questions)
        print(len(all_open))
        time.sleep(1.1)
    else:
        print('done')
        break

chunk starting 0
100
chunk starting 1
200
chunk starting 2
300
chunk starting 3
400
chunk starting 4
500
chunk starting 5
600
chunk starting 6
700
chunk starting 7
800
chunk starting 8
900
chunk starting 9
1000
chunk starting 10
1100
chunk starting 11
1200
chunk starting 12
1300
chunk starting 13
1400
chunk starting 14
1500
chunk starting 15
1600
chunk starting 16
1700
chunk starting 17
1800
chunk starting 18
1900
chunk starting 19
2000
chunk starting 20
2100
chunk starting 21
2200
chunk starting 22
2300
chunk starting 23
2399
chunk starting 24
done


In [8]:
joblib.dump(all_open, 'all_open.joblib')

['all_open.joblib']

In [9]:
questions = joblib.load('all_open.joblib')

In [10]:
questions = [x for x in questions if x.num_forecasters > 40 and type(x) != forecasting_tools.data_models.questions.DateQuestion]

In [11]:
set([type(x) for x in questions])

{forecasting_tools.data_models.questions.BinaryQuestion,
 forecasting_tools.data_models.questions.MultipleChoiceQuestion,
 forecasting_tools.data_models.questions.NumericQuestion}

In [12]:
binaries = [x for x in questions if type(x) == forecasting_tools.data_models.questions.BinaryQuestion]

In [13]:
binaries = [x for x in binaries if x.community_prediction_at_access_time is not None]

In [14]:
len(binaries)

699

In [15]:
mc = [x for x in questions if type(x) == forecasting_tools.data_models.questions.MultipleChoiceQuestion]

In [16]:
x = mc[0]

In [17]:
mc = [x for x in mc if x.api_json['question']['aggregations']['recency_weighted']['latest'] is not None]

In [18]:
len(mc)

25

In [19]:
num = [x for x in questions if type(x) == forecasting_tools.data_models.questions.NumericQuestion]

In [20]:
len(num)

142

In [21]:
num = [x for x in num if x.api_json['question']['aggregations']['recency_weighted']['latest'] is not None]

In [22]:
len(num)

142

In [23]:
questions = binaries + mc + num

In [24]:
len(questions)

866

In [25]:
!mkdir open

mkdir: cannot create directory ‘open’: File exists


In [26]:
from tqdm import tqdm

In [27]:
for question in tqdm(questions):
    fn=f'open/{question.id_of_question}.joblib'
    joblib.dump(question, fn)

100%|████████████████████████████████████████| 866/866 [00:01<00:00, 804.76it/s]


In [100]:
question = joblib.load('open/37229.joblib')